[Reference](https://medium.com/shelf-io-engineering/how-did-we-make-postgres-perform-reads-2x-faster-with-partitioning-1a286f25cbbd)

```
CREATE TABLE events.events
(
    customer_id             varchar(16)  not null,
    event_id                varchar(36)  not null,
    created_at timestamp    not null,
    --other fields
    constraint unique_customer_event
        unique (customer_id, event_id, created_at)
) PARTITION BY RANGE (created_at);
```

```
CREATE TABLE IF NOT EXISTS events.default_events_partition PARTITION OF events.events_partitioned_by_date DEFAULT;
ALTER TABLE events.default_events_partition
    ADD PRIMARY KEY (customer_id, event_id);

CREATE TABLE IF NOT EXISTS events.events_2020_01 PARTITION OF events.events FOR VALUES FROM ('2020-01-01') TO ('2020-02-01');
ALTER TABLE events.events_2020_01
    ADD PRIMARY KEY (customer_id, event_id);
--- repeat command bellow changing partition dates and ranges
```

```
DO $$
BEGIN
    IF NOT EXISTS (SELECT FROM pg_class WHERE relname  = 'events_' || to_char(now() + interval '1 month', 'YYYY_MM'))
    THEN
        EXECUTE format(
            'CREATE TABLE IF NOT EXISTS events.events_%s PARTITION OF events.events FOR VALUES FROM (''%s'') TO (''%s'');' ||
            'ALTER TABLE events.events_%s ADD PRIMARY KEY (customer_id, event_id);',
            to_char(now() + interval '1 month', 'YYYY_MM'),
            to_char(date_trunc('month', now() + interval '1 month'), 'YYYY-MM-DD'),
            to_char(date_trunc('month', now() + interval '2 months'), 'YYYY-MM-DD'),
            to_char(now() + interval '1 month', 'YYYY_MM')
        );
    END IF;
END $$;
```

```
begin;
alter table events.events
    rename to old_events;
alter table events.events_partitioned_by_date
    rename TO events;
commit;
```